![sql](img/sql-logo.jpg)

# A beginner's guide to databases, SQL, & using them with `pandas`

**Scenario:** You are a data analyst for the Homeland Security, trying to create reports on the active airports world wide. The data you need to access is in a SQL database. YOu need to be able to query for the data in a database!

## Learning goals:
- Goal 1: Summarize the use case for sql in the data science skill set
- Goal 2: Define key sql terminology
- Goal 3: Get information about DB schema and table structure
- Goal 4: Use basic SQL commands:
    - Construct SQL queries
    - Use JOIN to merge tables along logical columns
    - Grouping Data with SQL
- Goal 5: Query data from pandas dataframes using SQL
- Goal 6: Convert SQL to pandas

## Goal 1: Summarize

[Netflix has a great article](https://medium.com/netflix-techblog/notebook-innovation-591ee3221233) describing three different data roles at their company, their different needs, and their toolsets.

![netflix](img/netflix-data-roles.jpeg)

Examining that graphic, SQL shows up as one of the tools of the _Data Engineer_ 

Data Engineers provide the essential data architecture services that make data science possible.

![hierarchy](img/ai-hierachy.png)

[Source: Monica Rogati’s fantastic Medium post “The AI Hierarchy of Needs”
](https://hackernoon.com/the-ai-hierarchy-of-needs-18f111fcc007)

![etl](img/etl.png)

[img source: Jeff Hammerbacher’s slide from UC Berkeley CS 194 course ](https://bcourses.berkeley.edu/courses/1377158/pages/cs-194-16-introduction-to-data-science-fall-2015)

### What is a Relational Database? 

![rdb](img/relational-dbms-model.png)
[reference for image ](https://www.studytonight.com/dbms/database-model.php)

In [ ]:
#fact table (number, main information) v. dimensional table (category, way of dividing facts into groups)

### POPULAR RDBMS

- SQLite
- MySQL
- PostgreSql
- Oracle DB
- SQL Server

***
## Goal 2: Database terminology

### Relational Database Schema

![schema](img/MySQL_Schema_Music_Example.png)

[source of image](https://database.guide/what-is-a-database-schema/)

In [ ]:
#When a key is passed from one table to another, it's a secondary key.

### Table columns view
![table example](img/columns.png)

### Terminology

- Schema
- Primary Key
- Foreign Key
- Structured queries
- Views

***
### SQLite

![sqlite](img/SQLite-Python.jpg)

"SQLite is a C library that provides a lightweight disk-based database that doesn’t require a separate server process and allows accessing the database using a nonstandard variant of the SQL query language. Some applications can use SQLite for internal data storage. It’s also possible to prototype an application using SQLite and then port the code to a larger database such as PostgreSQL or Oracle." - [sqlite documentation](https://docs.python.org/2/library/sqlite3.html)



## But what about connecting to database servers?

To connect to an Oracle database server there is a different package, `cx_Oracle`. The documentation for that package is [here](https://oracle.github.io/python-cx_Oracle/)

***
## Goal 3: Get going with sqlite!

In [69]:
import sqlite3

 Volume in drive C is OS
 Volume Serial Number is 82A5-4D9B

 Directory of C:\Users\Rocio\Desktop\Class_files\dc-ds-071519\Module-2\week-1\day_1_sql_pandas



File Not Found


#### Load a DB object with `connect` and `cursor`

In [71]:
con = sqlite3.connect('flights.db')
cursor = con.cursor()

#### Use sqlite_master to find all the tables in the schema
Get the schema of a database from a db in sqlite

In [72]:
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
print(cursor.fetchall())
# Tables with actual information in the database

[('airports',), ('airlines',), ('routes',)]


In [6]:
cursor.description
#Tables of acutal database

(('name', None, None, None, None, None, None),)

In [9]:
cursor.execute("SELECT name, type From sqlite_master;")
print(cursor.fetchall())

[('airports', 'table'), ('ix_airports_index', 'index'), ('airlines', 'table'), ('ix_airlines_index', 'index'), ('routes', 'table'), ('ix_routes_index', 'index')]


#### Get information about one table

**A note about** `execute`<br>
Each time you use it, you reset the value of cursor

In [13]:
cursor.execute("SELECT * FROM airports as a").fetchone()
# Only shows one record, like head(1). .fetchall() shows all the data
# 'as a' assigns alias

(0,
 '1',
 'Goroka',
 'Goroka',
 'Papua New Guinea',
 'GKA',
 'AYGA',
 '-6.081689',
 '145.391881',
 '5282',
 '10',
 'U',
 'Pacific/Port_Moresby')

#### Use description

In [20]:
cursor.description
# Gives name of each column

(('index', None, None, None, None, None, None),
 ('id', None, None, None, None, None, None),
 ('name', None, None, None, None, None, None),
 ('city', None, None, None, None, None, None),
 ('country', None, None, None, None, None, None),
 ('code', None, None, None, None, None, None),
 ('icao', None, None, None, None, None, None),
 ('latitude', None, None, None, None, None, None),
 ('longitude', None, None, None, None, None, None),
 ('altitude', None, None, None, None, None, None),
 ('offset', None, None, None, None, None, None),
 ('dst', None, None, None, None, None, None),
 ('timezone', None, None, None, None, None, None))

#### Or use `Pragma`
`Pragma` tool [link here](https://www.sqlite.org/pragma.html#pragma_table_info)

**output**<br>
`(column id, column name, data type, whether or not the column can be NULL, and the default value for the column)`

In [27]:
cursor.execute("PRAGMA table_info(airports)")
info = cursor.fetchall()
print(*info, sep = "\n") 

(0, 'index', 'INTEGER', 0, None, 0)
(1, 'id', 'TEXT', 0, None, 0)
(2, 'name', 'TEXT', 0, None, 0)
(3, 'city', 'TEXT', 0, None, 0)
(4, 'country', 'TEXT', 0, None, 0)
(5, 'code', 'TEXT', 0, None, 0)
(6, 'icao', 'TEXT', 0, None, 0)
(7, 'latitude', 'TEXT', 0, None, 0)
(8, 'longitude', 'TEXT', 0, None, 0)
(9, 'altitude', 'TEXT', 0, None, 0)
(10, 'offset', 'TEXT', 0, None, 0)
(11, 'dst', 'TEXT', 0, None, 0)
(12, 'timezone', 'TEXT', 0, None, 0)


#### Making fetch happen

`.fetchall()` is how you get the query results out of the object.

You can also `.fetchone()` or `.fetchmany()`

**Task:** Get the descriptive data for airlines and routes tables

In [28]:
cursor.execute("PRAGMA table_info(airlines)")
info = cursor.fetchall()
print(*info, sep = "\n")

(0, 'index', 'INTEGER', 0, None, 0)
(1, 'id', 'TEXT', 0, None, 0)
(2, 'name', 'TEXT', 0, None, 0)
(3, 'alias', 'TEXT', 0, None, 0)
(4, 'iata', 'TEXT', 0, None, 0)
(5, 'icao', 'TEXT', 0, None, 0)
(6, 'callsign', 'TEXT', 0, None, 0)
(7, 'country', 'TEXT', 0, None, 0)
(8, 'active', 'TEXT', 0, None, 0)


In [30]:
cursor.execute("PRAGMA table_info(routes)")
info = cursor.fetchall()
print(*info, sep = "\n") 

(0, 'index', 'INTEGER', 0, None, 0)
(1, 'airline', 'TEXT', 0, None, 0)
(2, 'airline_id', 'TEXT', 0, None, 0)
(3, 'source', 'TEXT', 0, None, 0)
(4, 'source_id', 'TEXT', 0, None, 0)
(5, 'dest', 'TEXT', 0, None, 0)
(6, 'dest_id', 'TEXT', 0, None, 0)
(7, 'codeshare', 'TEXT', 0, None, 0)
(8, 'stops', 'TEXT', 0, None, 0)
(9, 'equipment', 'TEXT', 0, None, 0)


***
## Goal 4: Use basic SQL commands 
- Construct SQL queries
- Use JOIN to merge tables along logical columns
- Grouping Data with SQL

### Construct SQL queries

In [ ]:
# From is the table.
# Select is the columns you want.
# Where is the conditional. (Ex. select IDS from Root Table where source is Guatamala city)
# Order by: how to group

**Select**

**From**

**Where**

**Order by**

**Limit**

#### Options for each:

**Select**:  `distinct`, using `as` to rename columns, single number aggregates

**From:** also uses aliasing with `as`

**Where**: equals is only one `=`, `between`, `in`, wildcards `%`, `AND`, `OR`, `NOT`

**Order by**: `asc` and `desc`

**Limit**:  #

**Task**: 
- Select only active airlines in the UK from the airlines table
- Select the unique list of countries with airports

In [34]:
cursor.execute("""SELECT distinct country FROM airlines""").fetchall()

[(None,),
 ('United States',),
 ('South Africa',),
 ('United Kingdom',),
 ('Russia',),
 ('Thailand',),
 ('Canada',),
 ('Australia',),
 ('Singapore',),
 ('Belgium',),
 ('Mexico',),
 ('Spain',),
 ('France',),
 ('United Arab Emirates',),
 ('Republic of Korea',),
 ('Pakistan',),
 ('Libya',),
 ('Gambia',),
 ('Ivory Coast',),
 ('Ukraine',),
 ('Democratic Republic of the Congo',),
 ('Iran',),
 ('Finland',),
 ('Brazil',),
 ('Colombia',),
 ('AEROCENTER',),
 ('Ghana',),
 ('Kenya',),
 ('Liberia',),
 ('Togo',),
 ('Somali Republic',),
 ('Morocco',),
 ('Canadian Territories',),
 ('Dominican Republic',),
 ('Japan',),
 ('Albania',),
 ('Nigeria',),
 ('Germany',),
 ('Slovenia',),
 ('Czech Republic',),
 ('Benin',),
 ('AEROCESAR',),
 ('Greece',),
 ('Chile',),
 ('Tanzania',),
 ('Bolivia',),
 ('Italy',),
 ('Sweden',),
 ('Argentina',),
 ('Sierra Leone',),
 ('Indonesia',),
 ('Senegal',),
 ('Afghanistan',),
 ('Uganda',),
 ('Bosnia and Herzegovina',),
 ('Gabon',),
 ('Angola',),
 ('Uzbekistan',),
 ('Namibia',),


In [36]:
cursor.execute("""SELECT name FROM airlines WHERE active = 'Y' AND country = 'United Kingdom';""").fetchmany()

[('Astraeus',)]

### SQL Joins

SQL joins can be used to both **add** data to a table and **remove** data from a table. 

![venn](img/venn.png)

**Task** Write a query that will join the latitude and longitude data from the airports table to the information on the routes table

In [50]:
cursor.execute("""SELECT a.latitude, a.longitutde, r.index, r.airline, r.source, r.dest 
                    FROM airports as a
                    JOIN routes as r on r.source_id = a.id  """).fetchone()

OperationalError: near "index": syntax error

### Grouping statements

Combines `select` and `group by` when you want aggregates by values

`select` `min(x)` ... `max()`, `sum()`, etc

`group by x`

**Task**<br>
- Which countries have the highest amount of active airports?
- Which countries have the highest amount of inactive airports?
- What about airports by timezones?

In [58]:
cursor.execute("""
SELECT country, count(*) AS count
FROM airlines
WHERE active = 'Y'
GROUP BY country
ORDER BY count DESC
LIMIT 10
""")
print(*cursor.fetchall(), sep='\n')

('United States', 141)
('Russia', 72)
('United Kingdom', 40)
('Germany', 37)
('Canada', 34)
('Australia', 26)
('China', 25)
('Spain', 24)
('Brazil', 23)
('France', 22)


In [59]:
cursor.execute("""
SELECT country, count(*) AS count
FROM airlines
WHERE active = 'N'
GROUP BY country
ORDER BY count DESC
LIMIT 10
""")
print(*cursor.fetchall(), sep='\n')

('United States', 939)
('Mexico', 427)
('United Kingdom', 367)
('Canada', 284)
('Russia', 158)
('Spain', 142)
('France', 97)
('Germany', 94)
('South Africa', 81)
('Nigeria', 80)


In [64]:
cursor.execute("""
SELECT timezone, count(*) AS count
FROM airports
GROUP BY 1
ORDER BY 2 DESC
LIMIT 10
""")
print(*cursor.fetchall(), sep='\n')
# Group by 1 means group by first thing selected, timezone. Order by 2 is order by airports.

('America/New_York', 628)
('America/Chicago', 373)
('Europe/Berlin', 319)
('America/Anchorage', 258)
('Europe/Paris', 232)
('America/Los_Angeles', 226)
('Asia/Chongqing', 222)
('Europe/London', 193)
('America/Toronto', 159)
('Asia/Calcutta', 141)


AttributeError: 'sqlite3.Cursor' object has no attribute 'con'

## Goal 5: Using sql within pandas to filter

`.query()`

[query documentation](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.query.html)

In [62]:
# Get data for an example
import pandas as pd
shelter_data=pd.read_csv('https://data.austintexas.gov/api/views/9t4d-g238/rows.csv?accessType=DOWNLOAD')
shelter_data.dtypes

Animal ID           object
Name                object
DateTime            object
MonthYear           object
Date of Birth       object
Outcome Type        object
Outcome Subtype     object
Animal Type         object
Sex upon Outcome    object
Age upon Outcome    object
Breed               object
Color               object
dtype: object

In [63]:
max_data = shelter_data.query('Name == "Max"')
max_data.head()
#vs
shelter_data[shelter_data['Name']=="Max"]

,Animal ID,Name,DateTime,MonthYear,Date of Birth,Outcome Type,Outcome Subtype,Animal Type,Sex upon Outcome,Age upon Outcome,Breed,Color
126,A800791,Max,07/29/2019 06:07:00 PM,07/29/2019 06:07:00 PM,07/26/2017,Return to Owner,NaN,Dog,Intact Male,2 years,Pit Bull,Brown/White
262,A788916,Max,07/27/2019 05:16:00 PM,07/27/2019 05:16:00 PM,11/26/2018,Return to Owner,NaN,Dog,Neutered Male,7 months,Pit Bull Mix,Brown Brindle/White
290,A799870,Max,07/26/2019 07:17:00 PM,07/26/2019 07:17:00 PM,07/13/2017,Return to Owner,NaN,Dog,Intact Male,2 years,German Shepherd,Brown/Black
1170,A799881,Max,07/13/2019 02:33:00 PM,07/13/2019 02:33:00 PM,05/13/2019,Transfer,Partner,Dog,Intact Male,1 month,Yorkshire Terrier Mix,Brown/White
1535,A798950,Max,07/07/2019 01:31:00 PM,07/07/2019 01:31:00 PM,07/02/2017,Adoption,NaN,Dog,Neutered Male,2 years,American Bulldog,White/Blue
1684,A799025,Max,07/05/2019 11:13:00 AM,07/05/2019 11:13:00 AM,01/19/2019,Return to Owner,NaN,Dog,Neutered Male,5 months,German Shepherd Mix,Tan
1964,A798743,Max,06/30/2019 04:54:00 PM,06/30/2019 04:54:00 PM,10/29/2018,Return to Owner,NaN,Dog,Intact Male,8 months,Maltese/Miniature Poodle,White/Tan
2419,A700767,Max,06/25/2019 02:12:00 PM,06/25/2019 02:12:00 PM,04/19/2011,Return to Owner,NaN,Dog,Neutered Male,8 years,Labrador Retriever/Plott Hound,Brown Brindle/White
2729,A797355,Max,06/21/2019 06:39:00 PM,06/21/2019 06:39:00 PM,12/12/2017,Adoption,NaN,Dog,Neutered Male,1 year,Pit Bull Mix,Chocolate/White
2741,A797407,Max,06/21/2019 04:05:00 PM,06/21/2019 04:05:00 PM,03/13/2019,Adoption,NaN,Dog,Neutered Male,3 months,German Shepherd/Chinese Sharpei,Tricolor


In [65]:
shelter_data.rename(index=str, columns={"Animal Type": "animal_type"}, inplace = True)
test = shelter_data.query('animal_type == "Dog"')
test.head()

,Animal ID,Name,DateTime,MonthYear,Date of Birth,Outcome Type,Outcome Subtype,animal_type,Sex upon Outcome,Age upon Outcome,Breed,Color
1,A800473,Bruce,07/31/2019 06:19:00 PM,07/31/2019 06:19:00 PM,07/21/2018,Adoption,NaN,Dog,Neutered Male,1 year,Labrador Retriever/German Shepherd,Black/Tan
7,A799309,*Kermit,07/31/2019 05:41:00 PM,07/31/2019 05:41:00 PM,07/07/2017,Adoption,NaN,Dog,Neutered Male,2 years,Pit Bull,White/Brown
8,A799783,Luna,07/31/2019 05:29:00 PM,07/31/2019 05:29:00 PM,05/27/2019,Adoption,NaN,Dog,Spayed Female,2 months,Labrador Retriever Mix,Tan/Brown
9,A800528,NaN,07/31/2019 04:54:00 PM,07/31/2019 04:54:00 PM,07/22/2018,Adoption,NaN,Dog,Spayed Female,1 year,Miniature Schnauzer Mix,Gray/Tan
10,A800741,Fat Girl,07/31/2019 04:29:00 PM,07/31/2019 04:29:00 PM,07/25/2017,Adoption,NaN,Dog,Spayed Female,2 years,Soft Coated Wheaten Terrier Mix,Cream/Black


## Goal 6: Transfering from sqlite to pandas

In [66]:
conn = sqlite3.connect("flights.db")
df = pd.read_sql_query(";", conn)
df

TypeError: 'NoneType' object is not iterable

**Task**: 
Convert one of the earlier queries in the lesson to a pandas data frame

## Integration

The [Chinook database](https://github.com/lerocha/chinook-database) is a sample database, representing a digital media store.

You need to create a query that can rank tracks in term of popularity.

The name of the database is `Chinook_Sqlite.sqlite`

Database information:<br>
- How many tables are in the database?
- What's the primary key of each table?
- What foreign keys join the tables together?
- If you had to draw a schema of how the tables are connected, what would it look like?

In [73]:
con = sqlite3.connect('Chinook_Sqlite.sqlite')
cursor = con.cursor()

In [77]:
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
print(len(cursor.fetchall()))

11


In [92]:
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';").fetchall()

[('Album',),
 ('Artist',),
 ('Customer',),
 ('Employee',),
 ('Genre',),
 ('Invoice',),
 ('InvoiceLine',),
 ('MediaType',),
 ('Playlist',),
 ('PlaylistTrack',),
 ('Track',)]

In [91]:
cursor.description

(('cid', None, None, None, None, None, None),
 ('name', None, None, None, None, None, None),
 ('type', None, None, None, None, None, None),
 ('notnull', None, None, None, None, None, None),
 ('dflt_value', None, None, None, None, None, None),
 ('pk', None, None, None, None, None, None))

In [98]:
cursor.execute("PRAGMA table_info(customer)")
info = cursor.fetchall()
print(*info, sep = "\n")

(0, 'CustomerId', 'INTEGER', 1, None, 1)
(1, 'FirstName', 'NVARCHAR(40)', 1, None, 0)
(2, 'LastName', 'NVARCHAR(20)', 1, None, 0)
(3, 'Company', 'NVARCHAR(80)', 0, None, 0)
(4, 'Address', 'NVARCHAR(70)', 0, None, 0)
(5, 'City', 'NVARCHAR(40)', 0, None, 0)
(6, 'State', 'NVARCHAR(40)', 0, None, 0)
(7, 'Country', 'NVARCHAR(40)', 0, None, 0)
(8, 'PostalCode', 'NVARCHAR(10)', 0, None, 0)
(9, 'Phone', 'NVARCHAR(24)', 0, None, 0)
(10, 'Fax', 'NVARCHAR(24)', 0, None, 0)
(11, 'Email', 'NVARCHAR(60)', 1, None, 0)
(12, 'SupportRepId', 'INTEGER', 0, None, 0)


To answer the question:<br>
- What are the max and min dates in the Invoice table?
- What tables would you need to answer "what is your most popular track?"
- What values from each table?

## Reflection